In [1]:
import pandas as pd

import ee
from functools import partial
from traitlets import (
    Dict, HasTraits, Unicode, Any, link, observe
)
import ipyvuetify as v

import sepal_ui.sepalwidgets as sw
from sepal_ui.scripts import utils as su
from component.widget.custom_widgets import *


Styles()

ResizeTrigger()

In [2]:
ee.Initialize()

In [3]:

# Let the user select which is the LC/LU column, could be just code or with description
# Ask user if there is a description column
# if there is a description, load both columns
# From all the classes let the user to choose the corresponding ESA class


In [61]:
GREEN = v.theme.themes.dark.success
IPCC_CLASSES = {
    1: ['Forest', GREEN], 
    2: ['Grassland', GREEN], 
    3: ['Cropland', GREEN], 
    4: ['Wetland', GREEN], 
    5: ['Settlement', 'gray'],
    6: ['Other', 'gray']
}

In [6]:
class Flex(v.Flex, sw.SepalWidget):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

In [11]:
class Remap(v.SimpleTable):
    
    matrix = Dict({}).tag(sync=True)
    
    def __init__(self, code_field, *args, **kwargs):

        "Tile to display a remap values"
        
        self.dense = True
        
        self.code_field = code_field
        
        headers = ['Code', 'Ipcc']
        
        # Instantiate an empty dictionary with code as keys and empty values
        for code in self.code_field:
            self.matrix[code] = ''
        
        header = [
            v.Html(
                tag = 'tr', 
                children = (
                    [v.Html(tag = 'th', children = [h]) for h in headers]
                )
            )
        ]
        
        rows = [
            v.Html(tag='tr', children=[
                v.Html(tag = 'td', children=[str(code)]), 
                self.get_ipcc_class(code),
                
            ]) for code in self.code_field
        ]
        
        self.children = [v.Html(tag = 'tbody', children = header + rows)]
        
        # Create table
        super().__init__(*args, **kwargs)
    
    def store(self, code, change):
        
        self.matrix[code] = change['new']
        
        # Change color
        color = IPCC_CLASSES[change['new']['value']][1]
        change['owner'].background_color = color


    def get_ipcc_class(self, code):
        
        """Get a selector """        
        items = [{ 'value':k, 'text':v[0] } for k, v in IPCC_CLASSES.items()]
        
        select = v.Combobox(
            _metadata={'name':code}, 
            items=items, 
            v_model=None, 
            dense=True,
            hide_details=True
        )
        
        select.observe(partial(self.store, code), 'v_model')
        
        return select
        

In [109]:
class GeeSelector(v.Card):
    
    asset = Any('').tag(sync=True)
    code_col = Any('').tag(sync=True)

    
    def __init__(self, *args, **kwargs):
        
        
        super().__init__(*args, **kwargs)
        
        self.map = 0
        self.ee_asset = None
        self.w_mapper = None
        
        self.asset_selector = AssetSelect(
            label='Select a LC/LU GEE asset', 
            default_asset = 'users/dafguerrerom/FAO/LULC_2012_AOI'
        )
        
        self.w_code = v.Select(label='Select code column', class_='pr-4', v_model='')
        self.mapper_btn = sw.Btn('Get attributes', small=True)
        
        self.w_asset = Flex(
            _metadata = {'name':'columns'},
            class_='d-flex align-center mb-2',
            children=[self.w_code, self.mapper_btn]
        )
        
        # List of components whose could be hidden/showed
        
        self.components = {}
        
        # Define view
        self.children = [
            self.asset_selector,
            self.w_asset
        ]
        
        #Link traits
        
        link((self.asset_selector, 'v_model'), (self, 'asset'))
        link((self.w_code, 'v_model'), (self, 'code_col'))
        
        # Create Events
        
        self.mapper_btn.on_event('click', self._get_mapper_matrix)
        
        
    def _get_mapper_matrix(self, widget, event, data):
        
        component_name = 'mapper'
        # Hide previous loaded mappers
        if self.w_mapper: self._remove_widget(self.w_mapper, component_name)
        
        # Get code and description fields
        code_fields = self._get_fields(self.code_col)
        
        # Create mapper widget
        self.w_mapper = Remap(code_fields, 
                              dense=True, 
                              height='400px',
                              _metadata={'name':component_name})
        
        self.components[component_name] = self.w_mapper
        self._add_widget(self.w_mapper, component_name)

    def _hide_components(self):
        """Hide all possible componentes"""
        
        for component in self.components.values():
            su.hide_component(component)
    
    def _validate_asset(self, asset):
        try:
            self.ee_asset = ee.FeatureCollection(asset)
        except Exception as e:
            print(f'There was an error {e}')
    
    @observe('asset')
    def _get_items(self, change):
        asset = change['new']
        
        # Hide previous loaded components
        self._hide_components()
        self._validate_asset(asset)
        
        self.w_code.loading=True
        # Get columns of dataset
        columns = self._get_cols()
        
        # Fill widgets with column names
        self.w_code.items = columns
        
        self.w_code.loading=False
    
        
    def _add_widget(self, widget, widget_name):
        """Add widget at the end of the children view"""
        
        su.show_component(widget)
        
        widgets_name = [widget._metadata['name'] for widget in self.children if widget._metadata]      
            
        if widget_name not in widgets_name:
            self.children = self.children + [widget]
            
    def _remove_widget(self, widget, widget_name):
        """Add widget at the end of the children view"""
        
        widgets_name = [widget._metadata['name'] for widget in self.children if widget._metadata]      
            
        if widget_name in widgets_name:
            
            children = self.children
            children.remove(widget)
            
            self.children = children

    def _get_fields(self, col_name):
        
        return sorted(list(set(self.ee_asset.aggregate_array(col_name).getInfo())))

    def _get_cols(self):
        
        if self.ee_asset:
            columns = ee.Feature(self.ee_asset.first()).propertyNames().getInfo()
            return sorted([str(col) for col in columns if col not in ['system:index', 'Shape_Area']])
        

In [110]:
gee_selector = GeeSelector()

In [114]:
v.Card(children=[gee_selector], class_='pa-4')

Card(children=[GeeSelector(asset='users/dafguerrerom/FAO/LULC_2012_AOI', children=[AssetSelect(class_='my-5', …